###### This is the tutorial from [Learning Pytorch with Examples] 

[Learning Pytorch with Examples]: http://pytorch.org/tutorials/beginner/pytorch_with_examples.html

## Numpy example
 
 Numpy로도 딥러닝을 구현 할 수 있습니다. 예측한 pred_y와 x의 유클리디안 거리를 이용해서 학습을 하는, 하나의 hidden layer를 가지고 있고 bias는 없는, 'a' Fully-connected ReLU network입니다. 
 
  A fully-connected ReLU network with one hidden layer and no biases, trained to predict y from x using Euclidean error.

In [1]:
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

for t in range(500):
    # forward pass: compute prediected y 
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)
    
    # backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0* (y_pred-y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    

0 31564766.8156
1 27739761.3388
2 25225685.5075
3 21285278.6565
4 15886316.7054
5 10562524.574
6 6515530.38864
7 3953937.04502
8 2477825.77688
9 1653440.40078
10 1182657.04538
11 898844.503523
12 714479.045091
13 586059.319048
14 490962.415897
15 417255.61528
16 358326.496532
17 310210.929817
18 270236.901824
19 236687.771612
20 208209.966017
21 183885.687737
22 162979.971703
23 144925.230919
24 129296.833131
25 115692.221784
26 103784.181768
27 93329.8838088
28 84114.0428307
29 75957.470426
30 68726.8961258
31 62305.0552418
32 56583.5040845
33 51470.6059726
34 46892.1729184
35 42784.3073231
36 39100.8228448
37 35783.5281864
38 32791.5746403
39 30085.041388
40 27634.5156784
41 25415.6490546
42 23403.6519274
43 21571.3006611
44 19901.6940332
45 18377.6616202
46 16985.4054314
47 15711.9051566
48 14547.0281419
49 13479.2321287
50 12498.7385519
51 11598.0641686
52 10770.2086149
53 10007.6009995
54 9305.52512895
55 8657.90043667
56 8060.02144631
57 7508.06461322
58 6997.52230816
59 6525.111

## Pytorch example

같은 Network를 Pytorch로 만들어 본 예제 입니다. 가장 큰 Numpy와 PyTorch의 차이점은 GPU를 활용할 수 있는지 여부죠. Numpy는 CPU보다 50배 이상의 속도를 낼 수 있는 GPU를 활용할 수 없습니다. GPU를 사용하고 싶으면 Tensor에 `.cuda()`만 붙이면 됩니다.  

The biggest difference between a numpy array and a PyTorch Tensor is that a PyTorch Tensor can run on either CPU or GPU. To run operations on the GPU, just cast the Tensor to a cuda datatype. 

In [2]:
import torch 

dtype = torch.FloatTensor

N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in).type(dtype)
y = torch.randn(N, D_out).type(dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H).type(dtype)
w2 = torch.randn(H, D_out).type(dtype)

learning_rate = 1e-6

for t in range(500):
    # forward
    h = x.mm(w1)
    h_relu = h.clamp(min =0)
    y_pred = h_relu.mm(w2)
    
    # compute and print loss
    loss = (y_pred -  y).pow(2).sum()
    print(t, loss)
    
    # backprop
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 40335652.3141585
1 39513108.05447667
2 37787591.97517537
3 30059369.970895823
4 19091720.694198493
5 10079908.608228728
6 5118007.288646791
7 2843022.0864315815
8 1837642.0950671304
9 1345933.405582495
10 1064259.6234953552
11 876198.9602949098
12 736808.5020888106
13 627425.8134995904
14 538609.5559812672
15 465534.26483252645
16 404491.22342907026
17 352881.9537960914
18 309028.68849163153
19 271576.463242352
20 239364.72075812647
21 211552.44407359883
22 187457.09659951646
23 166506.9407603704
24 148234.313559541
25 132243.91209931485
26 118211.95519689145
27 105869.80177061999
28 94981.61498788383
29 85351.00310995778
30 76838.45980227028
31 69276.70223393833
32 62549.838822281454
33 56554.420365297
34 51200.04726609064
35 46412.233553943515
36 42124.14498501526
37 38275.972718509845
38 34818.876677427674
39 31708.352676499395
40 28906.967163681147
41 26384.932924495646
42 24106.533633792482
43 22045.5405253579
44 20181.66843279042
45 18496.624024882738
46 16967.612288592914
47 1

## Variables and autograd

A PyTorch Variable is a wrapper around a PyTorch Tensor, and represents a node in a computational graph. If x is a Variable then <u>__x.data__</u> is a Tensor __giving its value__, and <u>__x.grad__</u> is another Variable holding __the gradient of x__ with respect to some scalar value.

PyTorch Variables have the same API as PyTorch tensors: (almost) any operation you can do on a Tensor you can also do on a Variable; __the difference is that autograd allows you to automatically compute gradients.__

In [3]:
import torch
from torch.autograd import Variable

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

N, D_in, H, D_out = 64, 1000, 100, 10

x = Variable(torch.randn(N, D_in).type(dtype), requires_grad = False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad = False)

w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # forward
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data[0])
    
    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Variables with requires_grad=True.
    loss.backward()
    
    # Update weights using gradient descent.
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data
    
    # Manually zero the gradients after updating weights
    w1.grad.data.zero_()
    w2.grad.data.zero_()

0 37067204.0
1 36777912.0
2 42103876.0
3 43066400.0
4 33611192.0
5 18811662.0
6 8271086.0
7 3590913.25
8 1897674.375
9 1258962.375
10 963073.0
11 787571.0625
12 663248.5625
13 567008.25
14 489103.75
15 424704.75
16 370872.0
17 325521.125
18 286908.65625
19 253857.734375
20 225404.1875
21 200803.3125
22 179415.703125
23 160738.953125
24 144362.0625
25 129957.28125
26 117242.5
27 105983.015625
28 95988.40625
29 87092.8984375
30 79158.3984375
31 72064.3046875
32 65749.9921875
33 60085.95703125
34 54979.95703125
35 50373.98046875
36 46208.57421875
37 42442.421875
38 39026.50390625
39 35920.41796875
40 33092.65234375
41 30517.625
42 28169.25390625
43 26023.654296875
44 24060.9453125
45 22263.416015625
46 20614.994140625
47 19101.41796875
48 17710.966796875
49 16433.173828125
50 15256.3564453125
51 14171.7998046875
52 13171.619140625
53 12248.0087890625
54 11395.0400390625
55 10606.4384765625
56 9877.08203125
57 9202.7724609375
58 8578.2158203125
59 7999.94140625
60 7463.4091796875
61 6964.8

## TensorFlow: Static Graphs

 TensorFlow와 PyTorch의 큰 차이점은 '__computational graphs가 static인가, dynamic인가__' 입니다. TensorFlow는 Static Graphs이기 때문에 미리 다 그려놓고, Session을 열고 모델을 작동시켜야 합니다. 하지만 PyTorch는 모델의 구성을 작동을 시킬 때 dynamic하게 그릴 수 있죠. 제일 마지막 예시를 참고하세요.

One of the main differences between TensorFlow and PyTorch is that TensorFlow uses static computational graphs while PyTorch uses dynamic computational graphs.

In TensorFlow we first set up the computational graph, then execute the same graph many times.

In [ ]:
import tensorflow as tf
import numpy as np

# First we set up the computational graph:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create placeholders for the input and target data; these will be filled
# with real data when we execute the graph.
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

# Create Variables for the weights and initialize them with random data.
# A TensorFlow Variable persists its value across executions of the graph.
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

# Forward pass: Compute the predicted y using operations on TensorFlow Tensors.
# Note that this code does not actually perform any numeric operations; it
# merely sets up the computational graph that we will later execute.
h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

# Compute loss using operations on TensorFlow Tensors
loss = tf.reduce_sum((y - y_pred) ** 2.0)

# Compute gradient of the loss with respect to w1 and w2.
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

# Update the weights using gradient descent. To actually update the weights
# we need to evaluate new_w1 and new_w2 when executing the graph. Note that
# in TensorFlow the the act of updating the value of the weights is part of
# the computational graph; in PyTorch this happens outside the computational
# graph.
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

# Now we have built our computational graph, so we enter a TensorFlow session to
# actually execute the graph.
with tf.Session() as sess:
    # Run the graph once to initialize the Variables w1 and w2.
    sess.run(tf.global_variables_initializer())

    # Create numpy arrays holding the actual data for the inputs x and targets
    # y
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    for _ in range(500):
        # Execute the graph many times. Each time it executes we want to bind
        # x_value to x and y_value to y, specified with the feed_dict argument.
        # Each time we execute the graph we want to compute the values for loss,
        # new_w1, and new_w2; the values of these Tensors are returned as numpy
        # arrays.
        loss_value, _, _ = sess.run([loss, new_w1, new_w2],
                                    feed_dict={x: x_value, y: y_value})
        print(loss_value)

## PyTorch: nn

nn 패키지는 input으로 부터 output을 내는 하나의 뉴럴넷으로 모델을 생각할 수 있게 복잡한 뉴럴넷 Module들을 하나의 set으로 만들어 줄 수 있습니다. 학습가능한 weights들도 가지고 있고요. 

PyTorch autograd makes it easy to define computational graphs and take gradients, but raw autograd can be a bit too low-level for defining complex neural networks; this is where the nn package can help. The __nn package defines a set of Modules__, which you can think of as a neural network layer that has produces output from input and may have some trainable weights.

In [15]:
import torch
from torch.autograd import Variable

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)


# nn.Sequential is a Module which contains other Modules, 
# and applies them in sequence to produce its output. 
# Each Linear Module computes output from input using a
# linear function, and holds internal Variables for its weight and bias.

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(size_average=False)

learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])
    
    # Zero the gradients before running the backward pass.
    model.zero_grad()
    
    # backprop
    loss.backward()
    
    # Update the weights using gradient descent. Each parameter is a Variable, so
    # we can access its data and gradients like we did before.
    for param in model.parameters():
        param.data -= learning_rate * param.grad.data

0 727.4041748046875
1 670.4338989257812
2 621.7718505859375
3 579.870361328125
4 542.8980712890625
5 509.494140625
6 479.18408203125
7 451.5528869628906
8 426.0928039550781
9 402.5011291503906
10 380.51458740234375
11 359.76416015625
12 340.09515380859375
13 321.4697570800781
14 303.76611328125
15 286.8943176269531
16 270.8576354980469
17 255.64755249023438
18 241.1605987548828
19 227.3219451904297
20 214.17486572265625
21 201.68475341796875
22 189.7937469482422
23 178.4855194091797
24 167.76295471191406
25 157.6134033203125
26 147.98529052734375
27 138.90525817871094
28 130.3197784423828
29 122.21292114257812
30 114.5558853149414
31 107.33808135986328
32 100.55870056152344
33 94.1842041015625
34 88.1982421875
35 82.58589172363281
36 77.33965301513672
37 72.43427276611328
38 67.83100128173828
39 63.523406982421875
40 59.49240493774414
41 55.72052764892578
42 52.19571304321289
43 48.8908576965332
44 45.807334899902344
45 42.92582702636719
46 40.22911834716797
47 37.69515609741211
48 35.

## PyTorch: optim

 여태까지 예시에서 weight들을 일일히 수작업으로 update를 시켜줬는데요, optim package를 이용해서 자동으로 weights를 update할 수 있게 합시다. 보통 자주 쓰는, SGD, RMSProp, Adam등등이 포함되어 있습니다. 

Rather than manually updating the weights of the model as we have been doing, we use the optim package to define an __Optimizer that will update the weights for us__. The optim package defines many optimization algorithms that are commonly used for deep learning, including SGD+momentum, RMSProp, Adam, etc.

In [5]:
import torch
from torch.autograd import Variable

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

0 612.5390625
1 596.7669067382812
2 581.4266357421875
3 566.4747924804688
4 551.9630126953125
5 537.8717041015625
6 524.1765747070312
7 510.8863830566406
8 497.93975830078125
9 485.38311767578125
10 473.23150634765625
11 461.42578125
12 450.0104675292969
13 439.0106201171875
14 428.300537109375
15 417.9240417480469
16 407.8313293457031
17 397.9815979003906
18 388.350830078125
19 378.9653625488281
20 369.8157043457031
21 360.8554992675781
22 352.16778564453125
23 343.7235107421875
24 335.4874572753906
25 327.4519958496094
26 319.6493835449219
27 312.00201416015625
28 304.5221252441406
29 297.1965026855469
30 290.0544738769531
31 283.0833435058594
32 276.2730712890625
33 269.603271484375
34 263.0655517578125
35 256.6680603027344
36 250.4051971435547
37 244.2755126953125
38 238.2632598876953
39 232.38812255859375
40 226.63121032714844
41 220.993896484375
42 215.48512268066406
43 210.09918212890625
44 204.81263732910156
45 199.63958740234375
46 194.56719970703125
47 189.61370849609375
48 1

## PyTorch: Custom nn Modules

 여태껏 예시로 봐온 모델처럼 그 구성이 단순하지 않은, 복잡한 모델의 정의하기 위해서는, 아래와 같은 형태로 모델의 subclass들을 정의하고 사용합니다. **__init__** 함수와 __forward__함수는 필수입니다. 

This implementation __defines the model as a custom Module subclass__. Whenever you want a model more complex than a simple sequence of existing Modules you will need to define your model this way.

In [17]:
import torch
from torch.autograd import Variable


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 691.1053466796875
1 637.4067993164062
2 592.0670166015625
3 552.9415893554688
4 518.369384765625
5 487.8084716796875
6 460.3174743652344
7 434.97991943359375
8 411.81365966796875
9 390.3524169921875
10 370.1130065917969
11 351.13739013671875
12 333.2427978515625
13 316.41851806640625
14 300.4898681640625
15 285.3551025390625
16 270.80523681640625
17 256.896484375
18 243.62200927734375
19 230.9693603515625
20 218.89996337890625
21 207.34434509277344
22 196.32215881347656
23 185.8072052001953
24 175.7633819580078
25 166.18992614746094
26 157.04832458496094
27 148.282470703125
28 139.9516143798828
29 132.01560974121094
30 124.42718505859375
31 117.20441436767578
32 110.37244415283203
33 103.8996810913086
34 97.76529693603516
35 91.90098571777344
36 86.35858154296875
37 81.12770080566406
38 76.18933868408203
39 71.54643249511719
40 67.16910552978516
41 63.05654525756836
42 59.191246032714844
43 55.557373046875
44 52.14807891845703
45 48.95088577270508
46 45.95216751098633
47 43.137546539

## PyTorch: Control Flow + Weight Sharing

 위에서 Tensorflow와의 차이점으로 말한 dynamic graphs의 예시입니다. 모델을 실행시킬 때, 랜덤하게 hidden layer의 갯수를 정하게 하고, 같은 layer를 여러번 쓸 수 있기 때문에, 같은 weights를 공유하게 하는 식의 구성도 Pytorch는 가능합니다. 
 이는 lua Torch에서 한 번 정해진 graph는 한 번밖에 못썼던 방식에서의 큰 발전을 의미하기도 합니다. 

To showcase the power of PyTorch dynamic graphs, we will implement a very strange model: a fully-connected ReLU network that on each forward pass randomly chooses a number between 1 and 4 and has that many hidden layers, reusing the same weights multiple times to compute the innermost hidden layers.

In [18]:
import random
import torch
from torch.autograd import Variable


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 691.2542114257812
1 691.6785278320312
2 681.0726928710938
3 704.6624755859375
4 683.928955078125
5 682.7415771484375
6 627.8536376953125
7 676.1588745117188
8 671.9998168945312
9 667.6866455078125
10 500.88690185546875
11 658.2841796875
12 653.0742797851562
13 666.4641723632812
14 583.3334350585938
15 635.613525390625
16 660.8465576171875
17 355.9708557128906
18 545.4658203125
19 526.350341796875
20 647.5702514648438
21 475.2806396484375
22 445.3224182128906
23 630.2243041992188
24 380.1802062988281
25 345.0728454589844
26 207.31703186035156
27 273.0865783691406
28 237.8612518310547
29 436.3343505859375
30 171.1424560546875
31 157.5147705078125
32 351.56707763671875
33 146.9322509765625
34 135.50083923339844
35 100.69477844238281
36 113.88420104980469
37 98.8094253540039
38 223.28927612304688
39 68.86663055419922
40 64.85445404052734
41 332.7262268066406
42 158.3668975830078
43 60.764808654785156
44 59.86162185668945
45 158.99461364746094
46 72.87171936035156
47 89.9012680053711
48 4